# 타이타닉 모델 만들기

In [1]:
import pandas as pd 
import joblib

In [2]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

### 데이터셋

In [3]:
df_train = pd.read_csv('../static/data/titanic_train.csv')
df_train.head(3)

Survived  Pclass  Sex        Age  SibSp  Parch   Fare  Cabin  Embarked
0         0       3    0  24.000000      0      0   8.85      7         3
1         0       3    1  29.699118      8      2  69.55      7         3
2         1       2    0  30.000000      0      0  12.35      7         2

In [4]:
X_train = df_train.iloc[:, 1:].values
y_train = df_train.iloc[:, 0].values
X_train.shape, y_train.shape

((668, 8), (668,))

In [5]:
df_test = pd.read_csv('../static/data/titanic_test.csv')
df_test.head(3)

Survived  Pclass  Sex   Age  SibSp  Parch    Fare  Cabin  Embarked
0         0       3    0  20.0      0      0  8.6625      7         3
1         0       3    1  22.0      0      0  7.8958      7         3
2         0       3    1  33.0      0      0  7.8958      7         0

In [6]:
X_test = df_test.iloc[:, 1:].values
y_test = df_test.iloc[:, 0].values
X_test.shape, y_test.shape

((223, 8), (223,))

In [7]:
scaler = MinMaxScaler()
scaler.fit(df_train.iloc[:, 1:])

MinMaxScaler()

In [8]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled.shape, X_test_scaled.shape

((668, 8), (223, 8))

In [9]:
joblib.dump(scaler, '../static/model/titanic_scaler.pkl')

['../static/model/titanic_scaler.pkl']

### 1. 로지스틱 회귀

In [10]:
lr_clf = LogisticRegression()
lr_clf.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [11]:
params = {
    #'C': [0.1, 1, 5, 10]
    #'C': [0.05, 0.08, 0.1, 0.2, 0.5]
    'C': [0.01, 0.03, 0.05, 0.07]
}

In [12]:
grid_cv = GridSearchCV(lr_clf, param_grid=params, scoring='accuracy', cv=5)
grid_cv.fit(X_train_scaled, y_train)
print(f'최고 평균 정확도: {grid_cv.best_score_:.4f}')
print('최적 파라미터:', grid_cv.best_params_)

최고 평균 정확도: 0.8174
최적 파라미터: {'C': 0.05}


In [13]:
best_lr = grid_cv.best_estimator_
pred = best_lr.predict(X_test_scaled)
accuracy_score(y_test, pred)

0.8026905829596412

In [14]:
joblib.dump(best_lr, '../static/model/titanic_lr.pkl')

['../static/model/titanic_lr.pkl']

### 2. SVM

In [15]:
sv_clf = SVC()
sv_clf.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [16]:
params = {
    #'C': [0.1, 1, 5, 7, 10]
    #'C': [8, 10, 12, 15, 20]
    'C': [13, 14, 15, 16, 17, 18]
}

In [17]:
grid_cv = GridSearchCV(sv_clf, param_grid=params, scoring='accuracy', cv=5)
grid_cv.fit(X_train_scaled, y_train)
print(f'최고 평균 정확도: {grid_cv.best_score_:.4f}')
print('최적 파라미터:', grid_cv.best_params_)

최고 평균 정확도: 0.8309
최적 파라미터: {'C': 17}


In [18]:
best_sv = grid_cv.best_estimator_
pred = best_sv.predict(X_test_scaled)
accuracy_score(y_test, pred)

0.8071748878923767

In [19]:
joblib.dump(best_lr, '../static/model/titanic_sv.pkl')

['../static/model/titanic_sv.pkl']

### 3. Random Forest

In [20]:
rf_clf = RandomForestClassifier()
rf_clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [21]:
params = {
    #'max_depth': [4, 6, 8, 10],
    'max_depth': [7, 8, 9],
    'min_samples_split': [2, 3, 4]
    #'min_samples_split': [3, 4, 5, 6]
}

In [22]:
grid_cv = GridSearchCV(rf_clf, param_grid=params, scoring='accuracy', cv=5)
grid_cv.fit(X_train_scaled, y_train)
print(f'최고 평균 정확도: {grid_cv.best_score_:.4f}')
print('최적 파라미터:', grid_cv.best_params_)

최고 평균 정확도: 0.8324
최적 파라미터: {'max_depth': 8, 'min_samples_split': 3}


In [23]:
best_rf = grid_cv.best_estimator_
pred = best_rf.predict(X_test_scaled)
accuracy_score(y_test, pred)

0.8340807174887892

In [24]:
joblib.dump(best_rf, '../static/model/titanic_rf.pkl')

['../static/model/titanic_rf.pkl']

### Test

In [25]:
index = 100
test_data = (df_test.iloc[index, 1:].values).reshape(1,-1)
label = df_test.iloc[index, 0]
test_data, label

(array([[ 2. ,  0. , 29. ,  0. ,  0. , 10.5,  5. ,  3. ]]), 1)

In [26]:
new_scaler = joblib.load('../static/model/titanic_scaler.pkl')

In [27]:
test_scaled = new_scaler.transform(test_data)

In [28]:
pred_lr = best_lr.predict(test_scaled)
pred_sv = best_sv.predict(test_scaled)
pred_rf = best_rf.predict(test_scaled)

In [29]:
label, pred_lr[0], pred_sv[0], pred_rf[0]

(1, 1, 1, 1)

In [30]:
df_test.head(3)

Survived  Pclass  Sex   Age  SibSp  Parch    Fare  Cabin  Embarked
0         0       3    0  20.0      0      0  8.6625      7         3
1         0       3    1  22.0      0      0  7.8958      7         3
2         0       3    1  33.0      0      0  7.8958      7         0

In [31]:
tmp = df_test.iloc[index, 1:]

In [32]:
data =  []
int_index_list = [0, 1, 3, 4, 6, 7]
for i in range(8):
    if i in int_index_list:
        data.append(int(tmp[i]))
    else:
        data.append(tmp[i])
data

[2, 0, 29.0, 0, 0, 10.5, 5, 3]

In [33]:
a = dict(zip(df_test.columns[1:], data))
a

{'Pclass': 2,
 'Sex': 0,
 'Age': 29.0,
 'SibSp': 0,
 'Parch': 0,
 'Fare': 10.5,
 'Cabin': 5,
 'Embarked': 3}